#CoLA

## Preparing the Environment

In [ ]:
#Google Colab - Drive Mounting
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#Install missing library keras-nlp
!pip install -q keras-nlp

In [ ]:
#Import the libraries
import tensorflow as tf
import keras_nlp
from tensorflow import keras
import numpy as np
import os
import re
import string
import random
import pandas as pd

## Data Preprocessing and Parameters Initialization

In [ ]:
#Finetuning Parameters
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128

In [ ]:
# Download of the vocabulary from BERT: Bert-uncased
vocab_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)
#Initialization of the Word Tokenizer, with a given vocabulary and a sequence length
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab_file, sequence_length=SEQ_LENGTH,
)

Reference https://github.com/CyberZHG/keras-xlnet/blob/master/demo/GLUE/CoLA/cola.py#:~:text=tokens.append,(0)

In [ ]:
# Definition of the Class Data using keras Sequence format
class DataSequence(keras.utils.Sequence):
    '''
    Organize the data into: encoded text/ label 
    Overwrite the Keras.Utils.Sequence Class
    '''
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return (len(self.y) + FINETUNING_BATCH_SIZE - 1) // FINETUNING_BATCH_SIZE

    def __getitem__(self, index):
        s = slice(index * FINETUNING_BATCH_SIZE, (index + 1) * FINETUNING_BATCH_SIZE)
        return [item[s] for item in self.x], self.y[s]


def preprocess_data(path):
  '''
  Given a file path, read the file and organize the data from the file
  Return Keras.utils.Sequence object ready to use for training, with the format: tokens/classes
  '''
  tokens,classes=[], []
  with open(path) as reader:
        for line in reader:
            line = line.strip()
            parts = line.split('\t')
            tokens.append(tokenizer(parts[-1]))
            if parts[1]=='0': #Depending on the data formatting, it could be 0 or '0.0'
              classes.append(1)
            else:
              classes.append(0)
  tokens, classes = np.array(tokens), np.array(classes)
  return DataSequence([tokens], classes)

In [ ]:
train_path = 'path_to_train_set'
dev_path = 'path_to_dev_set'
#Load the training data
train_data = preprocess_data(train_path)
dev_data = preprocess_data(dev_path)

## Load the model and change the head

In [ ]:
# Load the pretrained model and display its structure
model = keras.models.load_model('path_to_the_pretrained_model',compile=False)
model.summary()

In [ ]:
# The model's head is modified for classification

inputs = keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32)
encoded_tokens = model(inputs)
x = keras.layers.GlobalAveragePooling1D()(encoded_tokens)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(2, activation="softmax")(x)


finetuning_model = keras.models.Model(inputs=inputs, outputs=outputs)
finetuning_model.summary()

## Model Training

In [ ]:
#Create tensorboard callback
logdir = "path_to_save_execution_information" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#Compile model 
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=FINETUNING_LEARNING_RATE),
    metrics=['sparse_categorical_accuracy'],
)

In [ ]:
#Model training
finetuning_model.fit(
    train_data, validation_data=dev_data, epochs=3,callbacks=[tensorboard_callback]
)

In [ ]:
# Add Tokenization layer to the model
inputs = keras.Input(shape=(), dtype=tf.string)
tokens = tokenizer(inputs)
outputs = finetuning_model(tokens)

#Save model
final_model = keras.Model(inputs, outputs)
final_model.save("path_to_save_model")

In [ ]:
# Restore the saved model
restored_model = keras.models.load_model("path_to_save_model", compile=False)

# Testing

In [ ]:
def preprocess_test(path):
    '''
    Given a file path, read the file and preprocess the data
    Return a numpy array of the test sentences
    '''
    tokens = [] 
    df = pd.read_csv(path, sep='\t', error_bad_lines=False)
    for _, row in df.iterrows():
        text_a = row['sentence']
        if not isinstance(text_a, str) :
            continue
        text = (text_a)
        encoded = ((text))
        tokens.append(encoded)
    tokens = np.array(tokens)
    return((tokens))

#Load test data
test_path = "path_to_test_data"
test_data = preprocess_test(test_path)

<ipython-input-17-8d06eb79ed68>:16: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  test_data = preprocess_test(test_path)


In [ ]:
#Generate predictions
pred = final_model.predict(test_data, batch_size=128,verbose=True).argmax(axis=-1)

9/9 [==============================] - 1s 53ms/step


In [ ]:
#Print the number of results
print(len(pred))

In [ ]:
#Format results into dataframe, ready to be uploaded on gluebenchmark.com
df = pd.DataFrame(results)

df.to_csv("CoLA.tsv",sep='\t', encoding='utf-8')

In [ ]:
#Load Tensorboard
%reload_ext tensorboard
%tensorboard --logdir="path_to_save_execution_information"

In [ ]:
#Code to automatically stop the run time for Google Colab
import time
time.sleep(60)
from google.colab import runtime
runtime.unassign()